# Generate xFG using defensive features as well


In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge

In [2]:
shots_df = pd.read_csv("../data/processed/def_variables.csv")
shots_df.head()


,GAME_ID,GAME_EVENT_ID,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,SHOT_MADE_FLAG,LOC_X,LOC_Y,PLAYER_ID,TEAM_ID,...,def_accel_mean,def_accel_max,shooter_speed_mean,shooter_speed_max,shooter_accel_mean,shooter_accel_max,window_frames,game_clock_release,shot_clock_release,release_idx
0,21500622,8,1,10,17,0,-25.0,219.0,201567,1610612739,...,3.020805,7.275966,16.081145,17.255065,3.380943,7.433706,26,617.03,21.72,164
1,21500622,10,1,10,10,1,168.0,178.0,202691,1610612744,...,5.548112,10.658684,9.376050,12.851462,10.286402,24.658259,26,610.24,24.00,334
2,21500622,28,1,8,36,0,-52.0,244.0,2544,1610612739,...,1.951771,4.857779,11.890170,12.715391,1.671228,3.731838,26,516.01,22.57,359
3,21500622,30,1,8,23,0,-135.0,214.0,203110,1610612744,...,4.650577,11.870451,10.710293,13.155977,6.231974,10.759305,26,503.01,23.17,309
4,21500622,34,1,8,6,0,22.0,51.0,101106,1610612744,...,4.505002,13.466623,13.653640,15.155399,3.536798,7.147216,26,486.02,22.94,163


### 1) Choose Columns


In [3]:
target = shots_df["SHOT_MADE_FLAG"] - shots_df["xFG_base"]

defense_columns = [
    "close_def_dist_release",
    "close_def_closing_speed_mean",
    "def_speed_mean",
    "def_accel_mean",
    "shooter_speed_mean",
    "shooter_accel_mean",
]

base_columns = ["xFG_base"]  # shot prior / shooter+location prior
feature_columns = base_columns + defense_columns


### 2) Define Learning Target
- "defense effect beyond expectation"  
- residual > 0 means shot went better than xFG expected

In [4]:
shots = shots_df.dropna(subset=["SHOT_MADE_FLAG", "xFG_base"]).copy()
shots["residual"] = shots["SHOT_MADE_FLAG"].astype(float) - shots["xFG_base"].astype(float)

### 3) Fit a smooth model of residual from (xFG_base + defense state)

In [5]:
X = shots[feature_columns]
y = shots["residual"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

residual_model = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="median")),
    ("scale", StandardScaler()),
    ("ridge", Ridge(alpha=1.0))
])

residual_model.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('impute', ...), ('scale', ...), ...]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"missing_values missing_values: int, float, str, np.nan, None or pandas.NA, default=np.nanThe placeholder for the missing values. All occurrences of`missing_values` will be imputed. For pandas' dataframes withnullable integer dtypes with missing values, `missing_values`can be set to either `np.nan` or `pd.NA`.",nan
,"strategy strategy: str or Callable, default='mean'The imputation strategy.- If ""mean"", then replace missing values using the mean along each column. Can only be used with numeric data.- If ""median"", then replace missing values using the median along each column. Can only be used with numeric data.- If ""most_frequent"", then replace missing using the most frequent value along each column. Can be used with strings or numeric data. If there is more than one such value, only the smallest is returned.- If ""constant"", then replace missing values with fill_value. Can be used with strings or numeric data.- If an instance of Callable, then replace missing values using the scalar statistic returned by running the callable over a dense 1d array containing non-missing values of each column... versionadded:: 0.20 strategy=""constant"" for fixed value imputation... versionadded:: 1.5 strategy=callable for custom value imputation.",'median'
,"fill_value fill_value: str or numerical value, default=NoneWhen strategy == ""constant"", `fill_value` is used to replace alloccurrences of missing_values. For string or object data types,`fill_value` must be a string.If `None`, `fill_value` will be 0 when imputing numericaldata and ""missing_value"" for strings or object data types.",None
,"copy copy: bool, default=TrueIf True, a copy of X will be created. If False, imputation willbe done in-place whenever possible. Note that, in the following cases,a new copy will always be made, even if `copy=False`:- If `X` is not an array of floating values;- If `X` is encoded as a CSR matrix;- If `add_indicator=True`.",True
,"add_indicator add_indicator: bool, default=FalseIf True, a :class:`MissingIndicator` transform will stack onto outputof the imputer's transform. This allows a predictive estimatorto account for missingness despite imputation. If 

### 4) Tight Guardness Feature

In [6]:
def add_guard_tightness_feature(
    df,
    model,
    dist_col="close_def_dist_release",
    eps=0.5,        # small distance perturbation
    scale=True
):
    df = df.copy()

    X = df[feature_columns].copy()

    # prediction at current distance
    pred_cur = model.predict(X)

    # prediction at slightly MORE space
    X_more = X.copy()
    X_more[dist_col] = X_more[dist_col] + eps
    pred_more = model.predict(X_more)

    # local sensitivity: how much worse defense gets per unit space
    # positive => giving space increases offensive advantage
    sensitivity = (pred_more - pred_cur) / eps

    # weight by shot danger
    guard_tightness = df["xFG_base"].values * sensitivity
    

    # optional rescale to something usable
    if scale:
        # robust scaling: map to roughly [0, 1]
        q1, q99 = np.nanpercentile(guard_tightness, [1, 99])
        if q99 > q1:
            guard_tightness = (guard_tightness - q1) / (q99 - q1)
            guard_tightness = np.clip(guard_tightness, 0.0, 1.0)

    df["guard_tightness"] = guard_tightness
    df["guard_tightness_rank"] = (df["guard_tightness"].rank(method="average", pct=True))   
    df["guard_sensitivity_raw"] = sensitivity
    return df


In [11]:
shots_with_feature = add_guard_tightness_feature(shots_df, residual_model)
shots_with_feature["guard_tightness_rank"].describe()
shots_with_feature["guard_tightness_rank"]


0     0.757895
1     0.694737
2     0.747368
3     0.810526
4     0.326316
        ...   
90    0.900000
91    0.900000
92    0.947368
93    0.189474
94    0.926316
Name: guard_tightness_rank, Length: 95, dtype: float64